# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# This is the csv file from part I of this assignment. 
file_one = "../WeatherPy/weatherpy_9_53.csv"
weather_df = pd.read_csv(file_one, encoding="ISO-8859-1")
weather_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,chenghai,CN,23.46,116.77,18.00,63.0,0.0,2.85,1.606849e+09
1,bluff,NZ,-46.60,168.33,14.44,52.0,100.0,1.79,1.606849e+09
2,rantepao,ID,-2.97,119.90,19.29,88.0,86.0,1.13,1.606849e+09
3,roma,US,43.21,-75.46,8.33,87.0,99.0,5.54,1.606849e+09
4,rikitea,PF,-23.12,-134.97,24.12,71.0,65.0,3.22,1.606849e+09
...,...,...,...,...,...,...,...,...,...
620,dvinskoy,RU,62.15,45.12,-3.43,96.0,96.0,3.24,1.606849e+09
621,sabzevar,IR,36.21,57.68,11.85,51.0,59.0,0.70,1.606849e+09
622,zihuatanejo,MX,17.63,-101.55,29.16,62.0,49.0,2.71,1.606849e+09
623,thohoyandou,ZA,-22.95,30.48,25.68,52.0,28.0,2.23,1.606849e+09


In [3]:
#Checking the data types to make sure I don't need to convert anything. 
weather_df.dtypes

City                    object
Country                 object
Latitude               float64
Longitude              float64
Max Temperature (F)    float64
Humidity (%)           float64
Cloudiness (%)         float64
Wind Speed (mph)       float64
Date                   float64
dtype: object

In [4]:
#Convert temperature to Farenheit.
weather_df['Max Temperature (F)'] = weather_df['Max Temperature (F)']*1.8+32
weather_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,chenghai,CN,23.46,116.77,64.400,63.0,0.0,2.85,1.606849e+09
1,bluff,NZ,-46.60,168.33,57.992,52.0,100.0,1.79,1.606849e+09
2,rantepao,ID,-2.97,119.90,66.722,88.0,86.0,1.13,1.606849e+09
3,roma,US,43.21,-75.46,46.994,87.0,99.0,5.54,1.606849e+09
4,rikitea,PF,-23.12,-134.97,75.416,71.0,65.0,3.22,1.606849e+09
...,...,...,...,...,...,...,...,...,...
620,dvinskoy,RU,62.15,45.12,25.826,96.0,96.0,3.24,1.606849e+09
621,sabzevar,IR,36.21,57.68,53.330,51.0,59.0,0.70,1.606849e+09
622,zihuatanejo,MX,17.63,-101.55,84.488,62.0,49.0,2.71,1.606849e+09
623,thohoyandou,ZA,-22.95,30.48,78.224,52.0,28.0,2.23,1.606849e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
weather_df = weather_df.dropna()
locations = weather_df[["Latitude", "Longitude"]]
humidity_map = weather_df["Humidity (%)"]

In [6]:
fig = gmaps.figure(center=(0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_map, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig
# Need to fix this map so the view doesn't repeat itself like this.

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
tmp = weather_df['Max Temperature (F)']
ws = weather_df['Wind Speed (mph)']
cld = weather_df['Cloudiness (%)']

vacation_df = weather_df.loc[(tmp>70)&(tmp<80) & 
                                        (ws<10) & (cld==0)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for idx in range(len(vacation_df["City"])):

    lat = vacation_df["Latitude"].iloc[idx]
    lng = vacation_df["Longitude"].iloc[idx]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
vacation_df["Hotel Name"] = hotels
vacation_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Hotel Name
18,hilo,US,19.73,-155.09,75.992,72.0,0.0,0.97,1.606849e+09,Hilo Hawaiian Hotel
46,atuona,PF,-9.80,-139.03,78.026,79.0,0.0,6.44,1.606849e+09,Villa Enata
48,rawson,AR,-43.30,-65.10,78.512,17.0,0.0,6.19,1.606849e+09,Hotel Punta León
67,raga,SS,8.46,25.68,75.650,21.0,0.0,1.94,1.606849e+09,Nearest hotel not found
87,mulege,MX,26.88,-111.98,76.496,19.0,0.0,4.28,1.606849e+09,Casa Granada
116,phan thiet,VN,10.93,108.10,74.120,83.0,0.0,5.14,1.606849e+09,Hotel Saigon-PT
128,avarua,CK,-21.21,-159.78,77.954,82.0,0.0,5.71,1.606849e+09,Paradise Inn
153,umm lajj,SA,25.02,37.27,73.418,49.0,0.0,2.66,1.606849e+09,Moon light Furnished Units
162,guerrero negro,MX,27.98,-114.06,75.794,19.0,0.0,1.65,1.606849e+09,"Casa Laguna, Bed & Breakfast"
185,kapaa,US,22.08,-159.32,74.534,71.0,0.0,0.80,1.606848e+09,Sheraton Kauai Resort at Coconut Beach


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Latitude", "Longitude"]]
humidity_map = vacation_df["Humidity (%)"]

In [10]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Hilo Hawaiian Hotel</dd>\n<dt>City</dt><dd>hilo</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Punta León</dd>\n<dt>City</dt><dd>rawson</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Nearest hotel not found</dd>\n<dt>City</dt><dd>raga</dd>\n<dt>Country</dt><dd>SS</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Casa Granada</dd>\n<dt>City</dt><dd>mulege</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Saigon-PT</dd>\n<dt>City</dt><dd>phan thiet</dd>\n<dt>Country</dt><dd>VN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Paradise Inn</dd>\n<dt>City</dt><dd>avarua</dd>\n<dt>Country</dt><dd>CK</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Moon light Furnished Units</dd>\n<dt>City</dt><dd>umm lajj</dd>\n<dt>Country</dt><dd>SA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Casa Laguna, 

In [11]:
# Add marker layer ontop of heat map
info_box_content=[f"Nearest Hotel: {hotel}" for hotel in hotels]
# hotel_info = 
markers = gmaps.marker_layer(locations, hotel_info)

fig = gmaps.figure(center=(0,0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_map, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))